In [15]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from PIL import ImageFont, ImageDraw, Image  # Import สำหรับจัดการภาษาไทย

# ------------------- ส่วนตั้งค่าโมเดลและ MediaPipe -------------------
# โหลดโมเดลที่เทรนแล้ว
try:
    model = load_model('sign_language_model.keras')
except:
    print("ไม่พบไฟล์ model กรุณาตรวจสอบ path ของไฟล์ .keras")
    exit()

# กำหนดรายการท่าทาง
actions = ['กลับ', 'ขอบคุณ', 'คุณสบายดีไหม', 'ช่วย', 'เชื่อ', 'แนะนำ', 'พา', 'รอ', 'สวัสดี', 'อะไร']

# โหลดฟอนต์ภาษาไทย (ต้องมีไฟล์ฟอนต์ในโฟลเดอร์เดียวกับไฟล์ python หรือระบุ path ให้ถูก)
# แนะนำให้ใช้ 'tahoma.ttf', 'angsana.ttc' หรือ 'THSarabunNew.ttf'
fontpath = "angsana.ttc" 
try:
    font = ImageFont.truetype(fontpath, 40) # ปรับขนาดตัวอักษรที่เลข 40
except IOError:
    print(f"ไม่พบฟอนต์ {fontpath} ระบบจะใช้ฟอนต์ default (อาจไม่แสดงภาษาไทย)")
    font = ImageFont.load_default()

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# ------------------- ฟังก์ชันตัวช่วย -------------------

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    # วาดจุดสำคัญบนใบหน้า
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    # วาดจุดสำคัญบนร่างกาย
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    # วาดจุดสำคัญบนมือซ้าย
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    # วาดจุดสำคัญบนมือขวา
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# ------------------- ส่วน Main Loop -------------------

cap = cv2.VideoCapture(1) # เปลี่ยนเป็น 0 หรือ 1 ตามกล้องที่มี

sequence = []
prediction = []
threshold = 0.8
text_to_display = "" # ตัวแปรสำหรับเก็บข้อความที่จะแสดง
 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # 1. ตรวจจับ MediaPipe
        image, results = mediapipe_detection(frame, holistic)
        
        # 2. วาด Landmarks
        draw_styled_landmarks(image, results)
        
        # 3. เตรียมข้อมูลและทำนายผล
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:] # เก็บ 30 เฟรมล่าสุด
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            prediction.append(np.argmax(res))
            
            # Logic การเลือกคำตอบที่เสถียร (ป้องกันภาพกระพริบไปมา)
            if np.unique(prediction[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    text_to_display = actions[np.argmax(res)]
        
        # 4. แสดงผลข้อความภาษาไทย (PIL Integration)
        if text_to_display:
            # แปลงภาพจาก OpenCV (BGR) เป็น PIL (RGB)
            image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(image_pil)
            
            # วาดพื้นหลังข้อความเพื่อให้เห็นชัดขึ้น (Optional)
            bbox = draw.textbbox((10, 50), text_to_display, font=font)
            draw.rectangle(bbox, fill=(0, 0, 0, 128)) # สีดำโปร่งแสง
            
            # วาดข้อความ
            draw.text((10, 50), text_to_display, font=font, fill=(0, 255, 0, 255)) # สีเขียว
            
            # แปลงกลับจาก PIL (RGB) เป็น OpenCV (BGR)
            image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
        
        # แสดงผลลัพธ์
        cv2.imshow('Thai Sign Language Detection', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━